# Examples using starlink-pywrapper: Introduction

This can be viewed and downloaded as an interactive jupyter notebook from https://github.com/Starlink/starlink-pywrapper/blob/master/doc/examples-notebook-intro.ipynb

This presents a guided set of examples running through some of the basics of using the `starlink-pywrapper` package to script Starlink Software Suite commands from Python. It uses the example file `scuba2_map.sdf` provided in http://ftp.eao.hawaii.edu/jcmt/usersmeetings/JCMT_tutorial_2018_Starlink_Analysis.tar.gz .

Starlink-pywrapper provides wrappers around the normal Starlink packages like KAPPA, CUPID etc, to make them easier to call from python, and so you don't have to use shell escapes in your arguments. It requires you to have a functioning Starlink Software Suite install on your computer and to know where Starlink is located, but it does not require you to run the Starlink setup scripts. It also packages up various parts of the help so that they are easily available within your python session.

This package provides wrappers for the commands in the Starlink packages KAPPA, CUPID, SMURF, POLPACK, PICARD, ATOOLS and FLUXES. There is also an (untested) wrapper around the Starlink Figaro package.


### Links:

 - http://starlink-pywrapper.readthedocs.io/en/latest/index.html
 - https://github.com/Starlink/starlink-pywrapper
 - http://starlink.eao.hawaii.edu



### Dependencies

This package depends on the `starlink-pyhds` package (which  is a low level package that allows you to read and write `.sdf` files within Python), and which itself depends on the `numpy` package. You can install this package and its dependencies with

`pip install starlink-pywrapper`

This package should work in either Python 2.7 or Python 3.5+.

## Contents

1. Setting up the package
1. Differences from standard Starlink
1. Basic usage
1. Getting help with the package
1. More advanced features
1. Running an ORAC-DR Reduction
1. Running many ORAC-DR reductions



## Setting up the package.

First of all, you need to import the package and let it know where Starlink is installed.

In [1]:
from starlink import wrapper

This warning message just means that I hadn't set the `$STARLINK_DIR` environmental variable, so the wrapper doesn't know where to find your installation of Starlink. If you had set the `$STARLINK_DIR` environmental variable before you started Python, then the wrapper would automatically use that Starlink installation.

We can manually tell the wrapper where Starlink is with:

In [4]:
wrapper.change_starpath('/star')

Change the path ``/star` to the path to your own Starlink installation.
You don't have to reimport the package after changing the `starpath`.

If you ever can't remember which Starlink you are using, look at the `wrapper.starpath`:

In [5]:
print(wrapper.starpath)

/star


### Differences from standard Starlink
This wrapper provides pythonic calling of Starlink commands, and is designed primarily for scripting. Therefore it has altered or removed some of the features you may be used to if you are experienced at using the command line Starlink.

1. Using this wrapper, Starlink will **not** remember your previously used values for any of the commands.
1. For keyword arguments, you have to give the full keyword name (shortening to a unique value isn't possible in this wrapper)
1. Starlink will not prompt you if you forget a keyword argument that is needed, the command will instead fail with an error.
1. Interactive features of some Starlink programs will not work.

### Unexpected *features* if you're used to Python

The way in which Starlink calls commands is very different from Python. The package attempts to neatly divide all the possible arguments for a command into `required` positional arguments and `optional` keyword arguments. However, often you will **have** to give some of the keyword arguments to run the command successfully. Hopefully the documentation on each command should make this clear. Occasionally there will be a required argument that you don't actually need -- normally you can safely give any value to these without causing any problems.

It is possible that the automatic generator script that creates the documentation and call signatures for the commands could have a bug such that the resulting `starlink-pywrapper` command cannot work succesfully. If you run into any issue like this, please let me know so I can fix this (s.graves AT eaobservatory.org).

## Basic usage of the commands.

The commands should be called like normal python functions. For example, to call the KAPPA `stats` function you would run (replace  `'Starlink_Analysis/scuba2_map.sdf'` with a path to an NDF file on your own machine):

In [6]:
from starlink import kappa
file =  'Starlink_Analysis/scuba2_map.sdf'
statsvals = kappa.stats(file)
print(statsvals)

skewness    29.107865318223343
maxcoord    [-1.3381681735838045, 0.021788981266010584, 0.00085]
 numgood    39948
  minwcs    18:53:46.073, 1:14:14.29, 0.00085
 minimum    -800.1018996019743
   order    False
  numbad    18133
   sigma    887.2211091888624
  numpix    58081
 maximum    48378.649881284786
    mean    95.57703057878183
     ndf    Starlink_Analysis/scuba2_map.sdf
  maxpos    [0, 0, 1]
    comp    DATA
  maxwcs    18:53:18.867, 1:14:54.30, 0.00085
mincoord    [-1.3361896752346525, 0.02159501271504439, 0.00085]
   total    3818111.2175611774
  minpos    [-102, -10, 1]
kurtosis    1147.2814951588975


Commands are easily called as `<package>.<commandname>(arguments, keyword1=...)`

Required (positional) arguments can be given either by position as above  or you can use the full name and pass the value as if it was a keyword. For example, you can pass the value `file` to the`ndf` argument of `kappa.stats` in either of these two ways:

In [7]:
statsvals = kappa.stats(file)
statsvals = kappa.stats(ndf=file)

Although it takes more typing, for complex command calls it is often clearer when you look back at a script if you use the second appraoch and give the argument name before each argument.

Optional (keyword) arguments are listed in the inline help (see the section `Getting help with the package` below), and are passed like normal Python keyword arguments. For example, the `order` and `comp` options can be changed from their defaults as:

In [8]:
statsvals = kappa.stats(file, order=True, comp='ERROR')
print(statsvals)

skewness    3.9059048292368095
maxcoord    [-1.337915996605891, 0.024154866836891307, 0.00085]
 numgood    39948
  median    21.865963058563082
  minwcs    18:53:19.667, 1:16:26.30, 0.00085
 minimum    10.638659143775554
   order    True
  numbad    18133
   sigma    50.65975753759802
  numpix    58081
 maximum    1157.3041341557991
    mean    41.05555477058478
     ndf    Starlink_Analysis/scuba2_map.sdf
  maxpos    [-13, 122, 1]
    comp    ERROR
  maxwcs    18:53:22.334, 1:23:02.30, 0.00085
mincoord    [-1.33810998137446, 0.022235009763934978, 0.00085]
   total    1640087.3019753105
  minpos    [-3, 23, 1]
kurtosis    26.029946463873603


### Returned values

The return object is a [`namedtuple`](https://docs.python.org/3/library/collections.html#collections.namedtuple) instance. The values in the object are automatically populated from the values written into the `$ADAM_DIR/<commandname>.sdf` parameter file. If you have used the `parget` command in Starlink to programmatically get return values this should be familiar to you -- the names will be the same.

You access fields in the output value as attributes:


In [9]:
print(statsvals.median, type(statsvals.median))

21.865963058563082 <class 'numpy.float64'>


To see the full list of fields programmatically, you can look at the `returnobj._fields` object.

In [10]:
print(statsvals._fields)

('skewness', 'maxcoord', 'numgood', 'median', 'minwcs', 'minimum', 'order', 'numbad', 'sigma', 'numpix', 'maximum', 'mean', 'ndf', 'maxpos', 'comp', 'maxwcs', 'mincoord', 'total', 'minpos', 'kurtosis')


This return value lets you use the calculated values of Starlink commands in your scripts.

Commands that primarly produce an output file rather than calculate values may not write anything very interesting to the return value; often it will just repeat the values of the parameters you set (and the default paramters you didn't give) from the command line call. For example, if we look at the output of  CONVERT's NDF2FITS command:

In [11]:
from starlink import convert

a = convert.ndf2fits(in_=file, out='!scuba2_map.fits')
print(a)

       in_    Starlink_Analysis/scuba2_map.sdf
       out    !scuba2_map.fits
    bitpix    0
   proexts    False
   profits    True
    native    False
  encoding    Auto
    duplex    False
 container    False
  checksum    True
      comp    A
  allowtab    True
provenance    None
    prohis    True


Unfortunately the `a.out` value is exactly as you passed the filename to `out` in the function call -- it does not represent the newly written file name itself. Here, the function call had a prepended exclamation mark, which allows you to overwrite a FITS file of the same name. The produced file on disk does not have that exclamation mark, but this is not shown to you in the `a.out` value.

In [12]:
import os
print(os.path.isfile(a.out), os.path.isfile('scuba2_map.fits'))

False True


You can also see that the parameter name 'in' from Starlink has been translated to `in_` for the Python wrapper; this is because `in` is a Python reserved name, so it has had an underscore attached to its name.

## Getting help with the package

This package attempts to bundle some of the most useful help with the package. You can see a short summary of a command and its python call signature and allowed keywords with the normal Python `help` function, or with the jupyter magic command `modulename.commandname?`. These are intended to be of use when you are running interactive sessions, either in the terminal or in the notebook.

In [13]:
from starlink import kappa, cupid
help(kappa.add)

Help on function add in module starlink.kappa:

add(in1, in2, out, **kwargs)
    Adds two NDF data structures.
    
    Runs the command: $KAPPA_DIR/add .
    
    Arguments
    ---------
    in1 : str,filename
        First input NDF
    
    in2 : str,filename
        Second input NDF
    
    out : str,filename
        Output NDF
    
    
    Keyword Arguments
    -----------------
    title : str
        Title for output NDF [!]
    
    
    Notes
    -----
    See http://www.starlink.ac.uk/cgi-bin/htxserver/sun95.htx/sun95.html?xref_ADD
    for full documentation of this command in the latest Starlink release



At the end of the help it should include a URL taking you to the full Starlink documentation of this command (not specific to using Python to call it.) This will include a lot more detail on the command and its options.

You can also see the module help -- this will include the version of starlink these wrappers were generated for, as well as the help (same as above) on all the commands in the module. Warning: this is very long.

In [14]:
help(cupid)

Help on module starlink.cupid in starlink:

NAME
    starlink.cupid - Runs commands from the Starlink CUPID package.

DESCRIPTION
    Autogenerated from the starlink .hlp and .ifl files,
    by starlink-pywrapper/helperscripts/generate_functions.py.
    
    Starlink version: 2018A
    b6ca36bf8884802759017298539489d11795861e (2018-07-06 09:36:39)

FUNCTIONS
    clumpinfo(ndf, **kwargs)
        Obtain information about one or more previously identified clumps.
        
        Runs the command: $CUPID_DIR/clumpinfo .
        
        Arguments
        ---------
        ndf : str,filename
            Input NDF containing clump identifications
        
        
        Keyword Arguments
        -----------------
        clumps : str
            The indices of the clumps to use [ALL]
        
        quiet : bool
            Supress screen output? [FALSE]
        
        
        Returns
        -------
        flbnd : List[float]
        
        fubnd : List[float]
        
        lbo

In order to make it easier to see what is available in a module, the package contains a `starhelp` command in the `utilities` subpackage. If you call this on a module this will show you a listing of all the commands available in that module, along with a short one line description of what it does.

In [15]:
from starlink.utilities import starhelp
starhelp(cupid)

clumpinfo     :     Obtain information about one or more previously identified clumps.
cupidhelp     :     Display information about CUPID.
extractclumps :     Extract previously identified clumps of emission from an NDF.
findback      :     Estimate the background in an NDF by removing small scale structure.
findclumps    :     Identify clumps of emission within a 1, 2 or 3 dimensional NDF.
makeclumps    :     Create simulated data containing clumps and noise.
outlineclump  :     Draw an outline around a 2-dimensional clump identified by CUPID.

If you call `starhelp` on a command, it will show you the **full** documentation of this Starlink command (not specific to this python wrapper) in .rst format. It will probably be better to look at the normal online Starlink documentation instead (linked at the end of the output seen running a normal `help` or `?` on a command).

In [16]:
starhelp(kappa.add)



ADD
===


Purpose
~~~~~~~
Adds two NDF data structures


Description
~~~~~~~~~~~
The routine adds two NDF data structures pixel-by-pixel to produce a
new NDF.


Usage
~~~~~


::

    
       add in1 in2 out
       



ADAM parameters
~~~~~~~~~~~~~~~



IN1 = NDF (Read)
````````````````
First NDF to be added.



IN2 = NDF (Read)
````````````````
Second NDF to be added.



OUT = NDF (Write)
`````````````````
Output NDF to contain the sum of the two input NDFs.



TITLE = LITERAL (Read)
``````````````````````
Value for the title of the output NDF. A null value will cause the
title of the NDF supplied for parameter IN1 to be used instead. [!]



Examples
~~~~~~~~
add a b c
This adds the NDF called b to the NDF called a, to make the NDF called
c. NDF c inherits its title from a.
add out=c in1=a in2=b title="Co-added image"
This adds the NDF called b to the NDF called a, to make the NDF called
c. NDF c has the title "Co-added image".



Notes
~~~~~


+ The output NDF contains the simple su

## More advanced features.

### Accessing the terminal output

If you've previously only examined the output of Starlink commands by reading or grepping the output written to the terminal, hopefully the return values shown here will be useful. Generally, you should not need to grep the string output of a Starlink command.


However, particularly while you are developing your script you may still sometimes want to see with the *normal* output, so this module does provide a means of doing that. The simplest way is to turn the standard python logging module into DEBUG mode. When you do that, the module will write both the full Starlink command it is running (useful if you run into problems) as well as the normal terminal output you would see if you ran this on the command line. For example, if you turn on DEBUG logging and run the `kappa.stats` command you will see:

In [17]:
import logging

logging.basicConfig(level=logging.DEBUG)
logging.root.setLevel(logging.DEBUG)

statsvals = kappa.stats(ndf=file, order=True, comp='ERROR')

DEBUG:starlink.wrapper:['/star/bin/kappa/stats', 'Starlink_Analysis/scuba2_map.sdf', 'order=True', 'comp=ERROR']
DEBUG:starlink.wrapper:
   Pixel statistics for the NDF structure /home/sgraves/PYTHON/starlink-pywrapper/doc/Starlink_Analysis/scuba2_map

      Title                     : G34.3
      NDF array analysed        : ERROR

         Pixel sum              : 1640087.30197531
         Pixel mean             : 41.0555547705848
         Standard deviation     : 50.659757537598
         Skewness               : 3.90590482923681
         Kurtosis               : 26.0299464638736
         Minimum pixel value    : 10.6386591437756
            At pixel            : (-3, 23, 1)
            Co-ordinate         : (18:53:19.667, 1:16:26.30, 0.00085)
         Maximum pixel value    : 1157.3041341558
            At pixel            : (-13, 122, 1)
            Co-ordinate         : (18:53:22.334, 1:23:02.30, 0.00085)
         Pixel median           : 21.8659630585631
         Total number of p

To turn this back to only showing INFO level logging information, you would run (in a jupyter notebook):

In [18]:
logging.root.setLevel(logging.INFO)

In a normal Python script or interactive session (not a jupyter notebook) you would normally run
```python
import logging
logger = logging.getLogger(__name__)
logger.setLevel('DEBUG')
```

#### Accessing the Standard Output as a string
Sometimes you may want to get the output that is normally written to the terminal as a string. You shouldn't normally need this, but if you do then pass the keyword argument `returnstdout=True` to any of the Starlink commands.

In [19]:
statsvals, stdout = kappa.stats(ndf=file, order=True, comp='ERROR', returnstdout=True)
print(stdout)


   Pixel statistics for the NDF structure /home/sgraves/PYTHON/starlink-pywrapper/doc/Starlink_Analysis/scuba2_map

      Title                     : G34.3
      NDF array analysed        : ERROR

         Pixel sum              : 1640087.30197531
         Pixel mean             : 41.0555547705848
         Standard deviation     : 50.659757537598
         Skewness               : 3.90590482923681
         Kurtosis               : 26.0299464638736
         Minimum pixel value    : 10.6386591437756
            At pixel            : (-3, 23, 1)
            Co-ordinate         : (18:53:19.667, 1:16:26.30, 0.00085)
         Maximum pixel value    : 1157.3041341558
            At pixel            : (-13, 122, 1)
            Co-ordinate         : (18:53:22.334, 1:23:02.30, 0.00085)
         Pixel median           : 21.8659630585631
         Total number of pixels : 58081
         Number of pixels used  : 39948 (68.8%)
         No. of pixels excluded : 18133 (31.2%)




This causes the command to return a tuple of the normal returned object and the Starlink terminal output as a string.


### Shell Escapes
Shell escapes are the sets of quotes or backslashes that you have to use when running Starlink in a shell if you have any special characters in your command line call -- they `escape` you from the shell's attempt to interpret those charaters.

Unlike when you call Starlink commands directly in your terminal, this python wrapper doesn't go through a shell, so you do not have to use shell escapes around string values. For example, when providing an NDF section to only include a 6 by 6 pixel square of your map you would do:

In [20]:
# No need to use shell escapes! E.g., if you want to pass an ndfsection you can do
section = '(0:5,0:5)'
statsvals = kappa.stats(ndf=file+section, comp='ERROR')
print(statsvals)
print(statsvals.ndf)

skewness    0.8710338530851028
maxcoord    [-1.3381875707353537, 0.02178898127010817, 0.00085]
 numgood    36
  minwcs    18:53:17.533, 1:15:14.30, 0.00085
 minimum    15.635780592588478
   order    False
  numbad    0
   sigma    43.44768072049565
  numpix    36
 maximum    145.2239807802179
    mean    56.78561067384343
     ndf    Starlink_Analysis/scuba2_map.sdf(0:5,0:5)
  maxpos    [1, 0]
    comp    ERROR
  maxwcs    18:53:18.600, 1:14:54.30, 0.00085
mincoord    [-1.338265159505499, 0.02188594394031615, 0.00085]
   total    2044.2819842583647
  minpos    [5, 5]
kurtosis    -0.8029823301673149
Starlink_Analysis/scuba2_map.sdf(0:5,0:5)


as opposed to running in your terminal, where you have to do:
```bash
stats "'../Starlink_Analysis/scuba2_map.sdf(0:5,0:5)'"
# or
stats ../Starlink_Analysis/scuba2_map.sdf\(0:5,0:5\)
```

Unfortunately you still can't include spaces in your NDF section (you also can't on the command line).

In [21]:
section = '(0:5, 0:5)'
statsvals = kappa.stats(ndf=file+section, comp='ERROR')

Exception: Starlink error occured during command:
/star/bin/kappa/stats ('Starlink_Analysis/scuba2_map.sdf(0:5, 0:5)',)
 stdout and stderr are appended below.
!! GRP1_PAREL: Un-matched delimiters '()' found in
!     'Starlink_Analysis/scuba2_map.sdf(0:5,'.
!  GRP_GRPEX: Unable to read names from group expression
!     Starlink_Analysis/scuba2_map.sdf(0:5,
!  Error obtaining a group of existing NDFs using group expression
!     "Starlink_Analysis/scuba2_map.sdf(0:5,"
!  Unable to associate a group of NDFs with parameter NDF.
!  STATS: Error computing simple statistics for an NDF's pixels.
!  Application exit status GRP__INVEL, Invalid element syntax given
!  Starlink_Analysis/scuba2_map.sdf(0:5, 0:5) comp=ERROR



### Error handling
The above example illustrates error handling. As well as the normal Python traceback seen at the top of the screen, you should also be shown the Starlink command that was run, its arguments, and the actual Starlink-generated error message. In this case, if you read the message you can see that it is trying to evaluate `'Starlink_Analysis/scuba2_map.sdf(0:5,'` as an NDF section, and of course failing. This is because the space we included in the NDF section causes Starlink to break the parameter at that point.

You can use normal Python exception handling to deal with these Starlink errors, in a normal ```try ... except ...``` block.

Currently ORAC-DR calls through `wrapper.oracdr` (see below) don't raise an error when they have problems; instead they produce a return value. This will probably be changed in the next version of the code.

## Accessing the FITS header

There is a utility function in the `starlink.utilities` module which will read in the FITS headers of an NDF file and convert it into an Astropy `fitsheader` object. This requires the `astropy` package to be installed and available to Python. It is used as:

In [23]:
from starlink.utilities import get_ndf_fitshdr
fitsheader = get_ndf_fitshdr(file)
fitsheader

TELESCOP= 'JCMT    '           / Name of Telescope                              
ORIGIN  = 'Joint Astronomy Centre, Hilo' / Origin of file                       
                                                                                
        ---- x,y,z triplet for JCMT relative to centre of earth ----            
ALT-OBS =               4120.0 / [m] Height of observatory above sea level      
LAT-OBS =      19.822838905884 / [deg] Latitude of Observatory                  
LONG-OBS=    -155.477027838737 / [deg] East longitude of observatory            
OBSGEO-X=    -5464587.04742954 / [m]                                            
OBSGEO-Y=    -2492998.88150847 / [m]                                            
OBSGEO-Z=     2150659.39661545 / [m]                                            
ETAL    =                 0.85 / Telescope efficiency                           
                                                                                
        ---- OMP and ORAC-DR